In [1]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
#ignore convergence warnings
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [3]:
# Download stock data for Apple (AAPL)
ticker = 'AAPL'
data = yf.download(ticker, start='2018-01-01', end='2023-01-01')

# Preview the data
print(data.head())

[*********************100%***********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close     Volume
Date                                                                        
2018-01-02  42.540001  43.075001  42.314999  43.064999  40.568943  102223600
2018-01-03  43.132500  43.637501  42.990002  43.057499  40.561863  118071600
2018-01-04  43.134998  43.367500  43.020000  43.257500  40.750271   89738400
2018-01-05  43.360001  43.842499  43.262501  43.750000  41.214230   94640000
2018-01-08  43.587502  43.902500  43.482498  43.587502  41.061142   82271200


In [4]:
# Use the 'Close' prices for the model
close_prices = data['Close']

In [5]:
# Split the data into training and testing sets
train = close_prices[:int(0.8 * len(close_prices))]
test = close_prices[int(0.8 * len(close_prices)):]

In [6]:
# After splitting into train and test, set frequency for the training index
train.index = pd.date_range(start=train.index[0], periods=len(train), freq='B')

In [7]:
ets_model = sm.tsa.ExponentialSmoothing(
    train, 
    trend='add', 
    seasonal='add', 
    seasonal_periods=365,
    initialization_method='estimated'  # or 'heuristic'
).fit(optimized=True)

In [8]:
# Forecast the test period
forecast_steps = len(test)
forecast = ets_model.forecast(steps=forecast_steps)
forecast_index = pd.date_range(start=train.index[-1] + pd.Timedelta(days=1), periods=forecast_steps, freq='B')
forecast = pd.Series(forecast, index=forecast_index)